# The fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## symplectic generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


# Goldman-Turaev bialgebra

## Linking of a pair of divisions

In [5]:
def ep(h1, h2)
  result = h1[:sign]*h2[:sign]
  result *= if h1[:letter].show == h2[:letter].show
    (h1[:index] < h2[:index]) ? h2[:sign] : h1[:sign]*(-1)
  else
    (Rseq.index(h1[:letter]) < Rseq.index(h2[:letter])) ? 1 : -1
  end
  return result
end
#--------------------------------------------

def lk(h1, h2) # h1={word: w1, div: div1}, h2={word: w2, div: div2})
  div_hashs = [h1,h2].map do |h|
    w = h[:word].flatten
    k = h[:div]
    s = w.size
    [{index: (k%s)+1, letter: w[k%s], sign: 1}, {index: k, letter: w[k-1].inverse, sign: -1}]
  end
  sum = 0
  2.times do |i|
    2.times{|j| sum += ep(div_hashs[0][i], div_hashs[1][j])}
  end
  return (-1)*(sum/2)
end; nil

## Goldman bracket

In [6]:
def bracket(w1, w2, verbose: false)
  """
  retern a Hash {divs: *, coeff: *, term: *, word: *}
  """
  #-----
  monomials = []
  if w1.show != '1' and w2.show != '1' 
    w1.size.times do |i|
      #--- divide two words each ---
      div1 = i+1
      facs = (w1.flatten.factors)*2
      nw1 = Word.new(facs[div1, w1.size])
      w2.size.times do |j|
        div2 = j+1
        #--- linking of the pair of divisions ---
        lk = lk({word: w1, div: div1}, {word: w2, div: div2})
        #---
        facs = (w2.flatten.factors)*2
        nw2 = Word.new(facs[div2, w2.size])
        #---
        monomials << {divs: [div1, div2], linking: lk, word: nw1*nw2}
      end
    end
  end

#-----
  key_order = %i[divs coeff term word]

  if verbose
    result = monomials
  else
    monomials.delete_if{|mono| mono[:linking] == 0}
    monomials.each{|mono| mono[:word] = mono[:word].cyclic_reduce}
    result = []
    while monomials.size > 0
      pop = monomials.pop
      pop[:divs] = [pop[:divs]]
      pop[:coeff] = pop[:linking]
      #---
      monomials.each do |mono|
        if mono[:word].conjugate?(pop[:word])
          pop[:coeff] += mono[:linking]
          pop[:divs] << mono[:divs]
          mono[:linking] = nil
        end
      end
      #---
      if pop[:coeff] != 0 or verbose
        pop.delete(:linkng)
        pop[:term] = pop[:word].show
        pop = pop.slice(*key_order)
        result << pop
      end
      #---
      monomials.delete_if{|mono| mono[:linking].nil?}
    end 
  end
  return (result.empty?) ? [{divs: [], coeff: 0, term: '1', word: Word.new}] : result
end; nil

In [7]:
w1 = Word.new(a[1], a[2], b[1].inverse)
w2 = Word.new(b[2], a[1], a[1].inverse) #, b[2])
printf [w1, w2].map(&:show).join(', ')  + "\n-------\n"

vbs = false #true #
bracket(w1, w2, verbose: vbs).each do |h|
  h.each do |k,v| 
    p "#{k}: #{v}" unless k == :word
  end
  printf "---\n"
end;
nil

acB, daA
-------
"divs: [[2, 1]]"
"coeff: 1"
"term: Bacd"
---


## Turaev cobraket

In [8]:
def cobracket(myw, verbose: false)
  """
  return a Hash {divs: *, coeff: *, term: *, words: *}
  """
  if myw.show == '1'
    monomials = []
  else
    #-----
    # 2つの div の組み合わせ全てについて、 {divs: *, linking: *, term: *, words: *} という Hash を生成。
    divs = (1..myw.size).to_a
    monomials = divs.combination(2).map do |combi|
      #--- linking of the pair of divisions ---
      lk = lk({word: myw, div: combi[0]},{word: myw, div: combi[1]})
      #--- divide the word --- 
      facs = myw.flatten.factors
      words = Array.new(2)
      words[0] = Word.new(facs[0..combi[0]-1]+facs[combi[1]..-1])
      words[1] = Word.new(facs[combi[0]..combi[1]-1])
      #--- get the wedge ---
      term = words.map(&:cyclic_reduce).join(" \u{2227} ")
      
      {divs: combi, linking: lk, term: term, words: words}
    end
  end
  #-----
  
  key_order = %i[divs coeff term words]
  
  if verbose
    result = monomials
  else
    monomials.delete_if{|mono| mono[:linking] == 0}
    monomials.each{|mono| mono[:words] = mono[:words].map(&:cyclic_reduce)}
    result = []
    while monomials.size > 0
      pop = monomials.pop
      pop[:divs] = [pop[:divs]]
      pop[:coeff] = pop[:linking]
      #---
      monomials.each do |mono|
        if mono[:words][0].conjugate?(pop[:words][0]) and mono[:words][1].conjugate?(pop[:words][1])
          pop[:coeff] += mono[:linking]
          pop[:divs] << mono[:divs]
          mono[:linking] = nil
        elsif mono[:words][0].conjugate?(pop[:words][1]) and mono[:words][1].conjugate?(pop[:words][0])
          pop[:coeff] += mono[:linking]*(-1)
          pop[:divs] << mono[:divs]
          mono[:linking] = nil
        end
      end
      #---
      if pop[:coeff] != 0 or verbose
        pop = pop.slice(*key_order)
        result << pop
      end
      #---
      monomials.delete_if{|mono| mono[:linking].nil?}
    end 
  end
  return (result.empty?) ? [{divs: [], coeff: 0, term: '0', words: [Word.new, Word.new]}] : result
end; nil

In [9]:
mw = Word.new(b[1], a[1], b[1]) #a1*Group.commutator(b[1], a[2]) #comms[1] #
printf mw.show + "\n-------\n"

vbs = false #true #
cobracket(mw, verbose: vbs).each do |h|
  h.each do |k,v| 
    p "#{k}: #{v}" unless k == :words
  end
  printf "---\n"
end; nil

bab
-------
"divs: []"
"coeff: 0"
"term: 0"
---


## Execution

In [10]:
a1, b1 = Word.new(a[1]), Word.new(b[1])
c = (a1*Word.new(b[1])*(a1.inverse)).flatten

samples = {
  s0: [
    Word.new,
    a1, 
    a1*Word.new(b[2]),
    a1*(Word.new(b[2]).inverse),
    a1*c*(c.inverse),
  ],
  s1: [
    a1^4,
    c,
    comms[1],
    Word.new(b[1])*a1*Word.new(b[1])
  ],
  s2: [
    a1*(comms[2].inverse),
    a1*(c.inverse)*(comms[2].inverse)*c,
    a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse),
    (a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse)).cyclic_reduce
  ],
  s3: [
    a1*comms[2],
    a1*(comms[2]^2),
    a1*comms[2]*c*comms[2]*(c.inverse),
    a1*(comms[2]^2)*Group.commutator(comms[2].inverse,c)
  ],
  s4: [
    a1*(Word.new(a[2])^3),
    a1*(Word.new(a[2], b[2].inverse)^3)
  ],
  s5: [
    a1*Group.commutator(a[2], a[3]),
    a1*(comms[2]*comms[3])*Group.commutator(a[2], a[3])*((comms[2]*comms[3]).inverse)
  ],
  s6: [
    a1*Group.commutator(b[1], a[2]),
    a1*(a1.inverse)*Word.new(b[1])*a1,
    c*(a1.inverse)*a1,
    c*(c.inverse),
    Word.new(a[1], b[1], b[1].inverse, b[2])
  ],
  s7: [
    a1*(Group.conjugate(a1, b1.inverse))*a1*(b1.inverse),
    a1*b1*((a1*(b1.inverse))^2),
    a1*Group.conjugate(b1.inverse, a1)*a1*(b1.inverse),
    (a1*Group.conjugate(b1.inverse, a1)*a1*(b1.inverse)).contract,
    a1*a1*(Word.new(b[2].inverse)^2)
    ]
  }

arr = samples #.slice(:s7) #:s0, :s6) #:s2) #, :s3) #:s0, :s1) #
arr.each do |k, v|
  printf "--- #{k} ---\n "
  v.each do |s|
    ws = s.show
    cob = cobracket(s, verbose: false).map do |h|
      coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
      coeff + h[:term]
    end
    #---
    puts "#{ws}  |--\u{03B4}-->  #{cob.join(' + ')}"
  end
end;nil

--- s0 ---
 1  |--δ-->  (0)0
a  |--δ-->  (0)0
a.d  |--δ-->  (-1)a ∧ d
a.D  |--δ-->  (0)0
a.abA.aBA  |--δ-->  1 ∧ a
--- s1 ---
 a.a.a.a  |--δ-->  aaa ∧ a + aa ∧ aa + a ∧ aaa
abA  |--δ-->  1 ∧ b
abAB  |--δ-->  (0)0
b.a.b  |--δ-->  (0)0
--- s2 ---
 a.dcDC  |--δ-->  (-1)a ∧ dcDC
a.aBA.dcDC.abA  |--δ-->  1 ∧ aBAdcDCab
a.dcDC.cdCDaBAdcDCabA  |--δ-->  (2)1 ∧ aBAdcDCab
aBAdcDCab  |--δ-->  (0)0
--- s3 ---
 a.cdCD  |--δ-->  (0)0
a.cdCD.cdCD  |--δ-->  acdCD ∧ cdCD
a.cdCD.abA.cdCD.aBA  |--δ-->  (0)0
a.cdCD.cdCD.dcDCabAcdCDaBA  |--δ-->  (0)0
--- s4 ---
 a.c.c.c  |--δ-->  acc ∧ c + ac ∧ cc
a.cD.cD.cD  |--δ-->  acDcD ∧ cD + acD ∧ cDcD
--- s5 ---
 a.ceCE  |--δ-->  (-1)aceC ∧ E + ace ∧ CE + (-1)ac ∧ C
a.cdCD.efEF.ceCE.feFEdcDC  |--δ-->  (-1)acdCDefEFceCfeFEdcDC ∧ E + acdCDefEFcefeFEdcDC ∧ CE + (-1)acdCDefEFcfeFEdcDC ∧ C
--- s6 ---
 a.bcBC  |--δ-->  (-1)abcB ∧ C + abc ∧ BC + (-1)abC ∧ cB
a.A.b.a  |--δ-->  (0)0
abA.A.a  |--δ-->  1 ∧ b
abA.aBA  |--δ-->  (0)0
abBd  |--δ-->  (-1)a ∧ d + (-1)ad ∧ 1
--- s7 --

# Conditions

## Involutivity for $[,]$ and $\delta$

$$[,]\circ \delta = 0$$

In [11]:
myw = samples[:s6][0] #4][-1] #
p myw.show
printf("=====\n")

vb = false #true #
##--------------
##  cobracket
cobracket(myw, verbose: vb).each do |h|
  h.each do |k,v|
    unless k == :words
      p "#{k}: #{v}"
    else
##--------------
##  bracket
      bracket(h[k][0], h[k][1], verbose: vb).each do |hh|
        hh.each do |kk,vv|
          unless kk == :word
            printf "\t--> "
            p "#{kk}: #{vv}"
          else
            nil
          end
        end
        printf("---\n")
      end
##--------------
    end
  end
  printf("---\n")
end; nil

"a.bcBC"
=====
"divs: [[4, 5]]"
"coeff: -1"
"term: abcB ∧ C"
	--> "divs: []"
	--> "coeff: 0"
	--> "term: 1"
---
---
"divs: [[3, 5]]"
"coeff: 1"
"term: abc ∧ BC"
	--> "divs: [[2, 1]]"
	--> "coeff: -1"
	--> "term: cabCB"
---
---
"divs: [[2, 4]]"
"coeff: -1"
"term: abC ∧ cB"
	--> "divs: [[3, 1]]"
	--> "coeff: -1"
	--> "term: abCBc"
---
---


## Compatibility condition for $[,]$ and $\delta$

$\forall v,w \in \pi$,
$$
\delta([v,w]) = w\cdot\delta(v)- v\cdot\delta(w),
$$
where 
$$
w\cdot(x\otimes y) = [w,x]\otimes y + x\otimes[w,y].
$$

In [12]:
w1 = samples[:s6][0] #Word.new(a[1], a[2])
w2 = samples[:s6][1] #Word.new(b[1])
p (w1*w2).show
printf("=====\n")

vb = false #true #
##--------------
##  bracket
bracket(w1, w2, verbose: vb).each do |h|
  h.each do |k,v|
    unless k == :word
      p "#{k}: #{v}"
    else
##--------------
##  cobracket
      cobracket(h[k], verbose: vb).each do |hh|
        hh.each do |kk,vv|
          unless kk == :words
            printf "\t--> "
            p "#{kk}: #{vv}"
          else
            nil
          end
        end
        printf("---\n")
      end
##--------------
    end
  end
  printf("---\n")
end; nil

"a.bcBC.a.A.b.a"
=====
"divs: [[4, 2]]"
"coeff: 1"
"term: Cabca"
	--> "divs: [[3, 5]]"
	--> "coeff: -1"
	--> "term: Cab ∧ ca"
---
	--> "divs: [[1, 5]]"
	--> "coeff: 1"
	--> "term: C ∧ abca"
---
	--> "divs: [[1, 4]]"
	--> "coeff: -1"
	--> "term: Ca ∧ abc"
---
---


# Experiments

# TODO

* [ ] cobracket を再帰的に作用させて、完全に分解することに何らかの意味があるか?
* [ ] $\ell_{2}$ の計算と cobracket との関係は? とくに両者にある division について。